# Problem 51
https://projecteuler.net/problem=51

In [1]:
println("Julia v", string(VERSION))

Julia v1.12.5


In [2]:
using Primes, Combinatorics, BenchmarkTools

In [3]:
function undigit(l::Array{Int64, 1})::Int64
    """

    julia> undigit([3, 4, 5])
    345
    
    """
    @assert all(n -> n ∈ 0:9, l) "Only integers in range 0:9 are valid inputs"
    s = 0 # sum
    for (i, n) in enumerate(reverse(l))
        s += n * 10^(i-1)
    end
    return s
end

@assert undigit([0, 1, 2]) == 12
@assert undigit([1, 2, 0]) == 120

In [4]:
function replace_digits(n, d, idx)
    """
    julia> replace_digits(9999, 0, [2, 3])
    9009
    """
    digits_of_n = reverse(digits(n))
    digits_of_n[idx] .= d
    return undigit(digits_of_n)
end

@assert replace_digits(111111, 9, [3, 4]) == 119911

### Solution 1

In [5]:
function prime_digit_replacements(treshold, n=1, info=false)
    n = 1 # start with primes of length n
    while true
        for p ∈ primes(10^(n-1), (10^n)-1) # generate primes of length n
            for index_combination ∈ powerset(1:n) # generate all index combinations in range 1:n
                valid_digits = 1 ∉ index_combination ? (0:9) : (1:9) # do not replace first digit with zero
                new_values = [replace_digits(p, d, index_combination) for d ∈ valid_digits]
                if sum(isprime, new_values) == treshold && p ∈ new_values
                    if info
                        new_primes = [x for x ∈ new_values if isprime(x)]
                        println("Values changed for positions: ", index_combination),
                        println("Prime family: ", new_primes)
                    end
                    return p
                end
            end
        end
        n += 1
    end
end

prime_digit_replacements(7, 1, true);
@assert prime_digit_replacements(7, 1) == 56003

Values changed for positions: [3, 4]
Prime family: [56003, 56113, 56333, 56443, 56663, 56773, 56993]


In [6]:
# info
prime_digit_replacements(8, 1, true);

Values changed for positions: [1, 3, 5]
Prime family: [121313, 222323, 323333, 424343, 525353, 626363, 828383, 929393]


In [7]:
# answer
@btime prime_digit_replacements(8)

  588.514 ms (24722370 allocations: 1.16 GiB)


121313

### Solution 2

In [8]:
function prime_digit_replacements_solution_two(treshold, n=1)
    for i in Iterators.countfrom(n), p ∈ primes(10^(i-1), (10^i)-1), index_combination ∈ powerset(1:i)
        new_values = [replace_digits(p, d, index_combination) for d ∈ (1 ∉ index_combination ? (0:9) : (1:9))]
        sum(isprime, new_values) == treshold && p ∈ new_values && return p
    end
end

@assert prime_digit_replacements_solution_two(7) == 56003

In [9]:
@btime prime_digit_replacements_solution_two(8)

  580.081 ms (24722370 allocations: 1.16 GiB)


121313